# Usage

In [1]:
#ignore the 3 lines below
import sys
pathnameto_eppy3000 = '../'
sys.path.append(pathnameto_eppy3000)

## Read the file
To use eppy3000 in a project:

In [2]:
from eppy3000.modelmaker import EPJ

fname = "../eppy3000/resources/snippets/V8_9/a.epJSON"
epj = EPJ(fname)

print(epj.epobjects['AirLoopHVAC'][0]) # print formattin is slightly broken



AirLoopHVAC                              !-  EP_KEY
    CRAC system                      !-  EPJOBJECT_NAME
    CRAC 1 Availability List             !-  availability_manager_list_name
    Air Loop Branches                    !-  branch_list_name
    Zone Equipment Inlet Node            !-  demand_side_inlet_node_names
    Zone Equipment Outlet Node           !-  demand_side_outlet_node_name
    8.5                                  !-  design_supply_air_flow_rate
    0                                    !-  idf_max_extensible_fields
    10                                   !-  idf_max_fields
    31                                   !-  idf_order
    Supply Inlet Node                    !-  supply_side_inlet_node_name
    Supply Outlet Node                   !-  supply_side_outlet_node_names


## Use the dot syntax
Or in more detail:

In [3]:
from eppy3000.modelmaker import EPJ
from io import StringIO

txt = """
{
    "Building": {
        "Bldg": {
            "idf_max_extensible_fields": 0,
            "idf_max_fields": 8,
            "idf_order": 3,
            "loads_convergence_tolerance_value": 0.05,
            "maximum_number_of_warmup_days": 30,
            "minimum_number_of_warmup_days": 6,
            "north_axis": 0.0,
            "solar_distribution": "MinimalShadowing",
            "temperature_convergence_tolerance_value": 0.05,
            "terrain": "Suburbs"
        }
    }
}
"""

sio = StringIO(txt)
epj = EPJ(sio)
abuilding = epj.epj.Building.Bldg
print(abuilding.solar_distribution)
print(abuilding.terrain)

MinimalShadowing
Suburbs


## EPJ.epobjects[key]
You can use idfobjects like in eppy:

the names ahve changed it is epobjects instead of idgobjects

In [4]:
buildings = epj.epobjects["Building"]
abuilding = buildings[0]
print(abuilding.terrain)

Suburbs


## Other EPJ functions
The following function also work:

- EPJ.save()
- EPJ.saveas()

## Accessing the IDD
Note that you have been opening the IDF file without referring to the IDD file. You can also open the IDF file with an IDD file. The code would look like this:

In [5]:
from eppy3000.modelmaker import EPJ
from pprint import pprint

epschemaname = "/Applications/EnergyPlus-8-9-0/Energy+.schema.epJSON"
fname = "../eppy3000/resources/snippets/V8_9/a.epJSON"
epj = EPJ(epjname=fname, epschemaname=epschemaname)
print(epj.epobjects['AirLoopHVAC'][0])


AirLoopHVAC                              !-  EP_KEY
    CRAC system                      !-  EPJOBJECT_NAME
    CRAC 1 Availability List             !-  availability_manager_list_name
    Air Loop Branches                    !-  branch_list_name
    Zone Equipment Inlet Node            !-  demand_side_inlet_node_names
    Zone Equipment Outlet Node           !-  demand_side_outlet_node_name
    8.5                                  !-  design_supply_air_flow_rate
    0                                    !-  idf_max_extensible_fields
    10                                   !-  idf_max_fields
    31                                   !-  idf_order
    Supply Inlet Node                    !-  supply_side_inlet_node_name
    Supply Outlet Node                   !-  supply_side_outlet_node_names


Now you have access to the IDD variables:

In [6]:
pprint(epj.epschema.epschemaobjects['AirLoopHVAC'].fieldnames())

['branch_list_name',
 'demand_side_outlet_node_name',
 'supply_side_outlet_node_names',
 'connector_list_name',
 'design_return_air_flow_fraction_of_supply_air_flow',
 'controller_list_name',
 'availability_manager_list_name',
 'demand_side_inlet_node_names',
 'supply_side_inlet_node_name',
 'design_supply_air_flow_rate']


You can look at the property of a particular fieldname:

In [7]:
pprint(epj.epschema.epschemaobjects['AirLoopHVAC'].fieldproperty('branch_list_name'))

{'data_type': 'object_list',
 'note': 'Name of a BranchList containing all the branches in this air loop',
 'object_list': ['BranchLists'],
 'type': 'string'}


You can also access the IDD for an IDF object from within the IDF object:

In [8]:
cracsystem = epj.epobjects['AirLoopHVAC'][0]
pprint(cracsystem.eppy_objepschema.fieldnames())
print()
pprint(cracsystem.eppy_objepschema.fieldproperty('demand_side_inlet_node_names'))


['branch_list_name',
 'demand_side_outlet_node_name',
 'supply_side_outlet_node_names',
 'connector_list_name',
 'design_return_air_flow_fraction_of_supply_air_flow',
 'controller_list_name',
 'availability_manager_list_name',
 'demand_side_inlet_node_names',
 'supply_side_inlet_node_name',
 'design_supply_air_flow_rate']

{'note': 'Name of a Node or NodeList containing the inlet node(s) supplying '
         'air to zone equipment.',
 'type': 'string'}


## Nested lists or arrays as fields
The old E+ had objects with a flat list of fields. As a result some objects needed repeating or extensible fields. An example of repeating/extensible fields are the coordinates in the object BuildingSurface:Detailed. These are the coordinates of the surface and the number of fields can vary depending on the shape of the surface.

The new JSON format treats the extensible fields as an array (an array in json and a list in python). Let us explore how to access and modify these list in eppy3000. Let us look at a single surface:

In [9]:
txt = """
{
    "BuildingSurface:Detailed": {
        "Zn001:Flr001": {
            "construction_name": "FLOOR",
            "idf_max_extensible_fields": 12,
            "idf_max_fields": 22,
            "idf_order": 27,
            "number_of_vertices": 4,
            "outside_boundary_condition": "Surface",
            "outside_boundary_condition_object": "Zn001:Flr001",
            "sun_exposure": "NoSun",
            "surface_type": "Floor",
            "vertices": [
                {
                    "vertex_x_coordinate": 15.24,
                    "vertex_y_coordinate": 0.0,
                    "vertex_z_coordinate": 0.0
                },
                {
                    "vertex_x_coordinate": 0.0,
                    "vertex_y_coordinate": 0.0,
                    "vertex_z_coordinate": 0.0
                },
                {
                    "vertex_x_coordinate": 0.0,
                    "vertex_y_coordinate": 15.24,
                    "vertex_z_coordinate": 0.0
                },
                {
                    "vertex_x_coordinate": 15.24,
                    "vertex_y_coordinate": 15.24,
                    "vertex_z_coordinate": 0.0
                }
            ],
            "view_factor_to_ground": 1.0,
            "wind_exposure": "NoWind",
            "zone_name": "Main Zone"
        }
    }
}"""

Let us open this as an EPJ():

In [10]:
from io import StringIO
from eppy3000.modelmaker import EPJ
from pprint import pprint

fhandle = StringIO(txt)
epj = EPJ(epjname=fhandle, epschemaname=epschemaname)
print(epj)



BuildingSurface:Detailed                         !-  EP_KEY
            Zn001:Flr001                         !-  EPJOBJECT_NAME
            FLOOR                                !-  construction_name
            12                                   !-  idf_max_extensible_fields
            22                                   !-  idf_max_fields
            27                                   !-  idf_order
            4                                    !-  number_of_vertices
            Surface                              !-  outside_boundary_condition
            Zn001:Flr001                         !-  outside_boundary_condition_object
            NoSun                                !-  sun_exposure
            Floor                                !-  surface_type
                                                 !-  vertices
                15.24                                !-  vertex_x_coordinate #1
                0.0                                  !-  vertex_y_coordinate 

Notice how the array items are inset. How dow we access the array items ?

Let us print the field names of `BuildingSurface:Detailed object:`

In [11]:
surfs = epj.epobjects["BuildingSurface:Detailed"]
surf = surfs[0]
pprint(surf.eppy_objepschema.fieldnames())

['surface_type',
 'number_of_vertices',
 'outside_boundary_condition_object',
 'construction_name',
 'wind_exposure',
 'vertices',
 'view_factor_to_ground',
 'zone_name',
 'sun_exposure',
 'outside_boundary_condition']


Notice the field vertices. Let us print and see what is in it:

In [12]:
pprint(surf.vertices)

[{'vertex_x_coordinate': 15.24,
  'vertex_y_coordinate': 0.0,
  'vertex_z_coordinate': 0.0},
 {'vertex_x_coordinate': 0.0,
  'vertex_y_coordinate': 0.0,
  'vertex_z_coordinate': 0.0},
 {'vertex_x_coordinate': 0.0,
  'vertex_y_coordinate': 15.24,
  'vertex_z_coordinate': 0.0},
 {'vertex_x_coordinate': 15.24,
  'vertex_y_coordinate': 15.24,
  'vertex_z_coordinate': 0.0}]


Now let is print one vertex:

In [13]:
pprint(surf.vertices[0])

{'vertex_x_coordinate': 15.24,
 'vertex_y_coordinate': 0.0,
 'vertex_z_coordinate': 0.0}


Looking at one coordinate:

In [14]:
print(surf.vertices[0].vertex_x_coordinate)

15.24


In [15]:
surf.vertices[0].vertex_x_coordinate = 88
surf.vertices.append(dict(vertex_x_coordinate=1.2,
                        vertex_y_coordinate=2.3,
                        vertex_z_coordinate=3.4))

How did our file change?

In [16]:
print(epj)


BuildingSurface:Detailed                         !-  EP_KEY
            Zn001:Flr001                         !-  EPJOBJECT_NAME
            FLOOR                                !-  construction_name
            12                                   !-  idf_max_extensible_fields
            22                                   !-  idf_max_fields
            27                                   !-  idf_order
            4                                    !-  number_of_vertices
            Surface                              !-  outside_boundary_condition
            Zn001:Flr001                         !-  outside_boundary_condition_object
            NoSun                                !-  sun_exposure
            Floor                                !-  surface_type
                                                 !-  vertices
                88                                   !-  vertex_x_coordinate #1
                0.0                                  !-  vertex_y_coordinate 

## Deleting, Copying and Creating idfobjects
The following functions are siilar to those in eppy.

### Creating new idfobjects
Let us start with a blank file:

In [17]:
from io import StringIO
from pprint import pprint
from eppy3000.modelmaker import EPJ


iddfname = "/Applications/EnergyPlus-8-9-0/Energy+.schema.epJSON"
epj = EPJ(epjname=StringIO("{}"), epschemaname=iddfname)

Now let us create a new `BuildingSurface:Detailed` object in it:

In [18]:
key = "BuildingSurface:Detailed"
objname = "wall1"
epj.newepobject(key, objname)
print(epj)


BuildingSurface:Detailed                         !-  EP_KEY
            wall1                                !-  EPJOBJECT_NAME
            Autocalculate                        !-  number_of_vertices
            WindExposed                          !-  wind_exposure
            Autocalculate                        !-  view_factor_to_ground
            SunExposed                           !-  sun_exposure


Notice how it put in all the default values. But what if we wanted to create the new object without the default values:

In [19]:
objname = "wall2"
epj.newepobject(key, objname, defaultvalues=False)
print(epj)


BuildingSurface:Detailed                         !-  EP_KEY
            wall1                                !-  EPJOBJECT_NAME
            Autocalculate                        !-  number_of_vertices
            WindExposed                          !-  wind_exposure
            Autocalculate                        !-  view_factor_to_ground
            SunExposed                           !-  sun_exposure

BuildingSurface:Detailed                         !-  EP_KEY
            wall2                                !-  EPJOBJECT_NAME


Wall2 does not include the default values. Now let us add more values using keyword arguments:

In [20]:
objname = "wall3"
lastobj = epj.newepobject(key, objname, defaultvalues=True,
                outside_boundary_condition="Surface",
                vertices=[{'vertex_x_coordinate': 15.24,
                            'vertex_y_coordinate': 0.0,
                            'vertex_z_coordinate': 0.0}])

print(epj)


BuildingSurface:Detailed                         !-  EP_KEY
            wall1                                !-  EPJOBJECT_NAME
            Autocalculate                        !-  number_of_vertices
            WindExposed                          !-  wind_exposure
            Autocalculate                        !-  view_factor_to_ground
            SunExposed                           !-  sun_exposure

BuildingSurface:Detailed                         !-  EP_KEY
            wall2                                !-  EPJOBJECT_NAME

BuildingSurface:Detailed                         !-  EP_KEY
            wall3                                !-  EPJOBJECT_NAME
            Autocalculate                        !-  number_of_vertices
            WindExposed                          !-  wind_exposure
            Autocalculate                        !-  view_factor_to_ground
            SunExposed                           !-  sun_exposure
            Surface                              !-  

## Deleting an idfobject
Deleting an idfobject is equally simple:

In [21]:
epj.removeepobject(key, "wall1")
print(epj)


BuildingSurface:Detailed                         !-  EP_KEY
            wall2                                !-  EPJOBJECT_NAME

BuildingSurface:Detailed                         !-  EP_KEY
            wall3                                !-  EPJOBJECT_NAME
            Autocalculate                        !-  number_of_vertices
            WindExposed                          !-  wind_exposure
            Autocalculate                        !-  view_factor_to_ground
            SunExposed                           !-  sun_exposure
            Surface                              !-  outside_boundary_condition
                                                 !-  vertices
                15.24                                !-  vertex_x_coordinate #1
                0.0                                  !-  vertex_y_coordinate #1
                0.0                                  !-  vertex_z_coordinate #1


How about copying an idfobject:

In [22]:
epj.copyepobject(key, "wall3", "wall4")
print(epj)



BuildingSurface:Detailed                         !-  EP_KEY
            wall2                                !-  EPJOBJECT_NAME

BuildingSurface:Detailed                         !-  EP_KEY
            wall3                                !-  EPJOBJECT_NAME
            Autocalculate                        !-  number_of_vertices
            WindExposed                          !-  wind_exposure
            Autocalculate                        !-  view_factor_to_ground
            SunExposed                           !-  sun_exposure
            Surface                              !-  outside_boundary_condition
                                                 !-  vertices
                15.24                                !-  vertex_x_coordinate #1
                0.0                                  !-  vertex_y_coordinate #1
                0.0                                  !-  vertex_z_coordinate #1

BuildingSurface:Detailed                         !-  EP_KEY
            wall4  

## Reading and writing IDF files
Eppy3000 works with JSON files. At the time of this documentation, Energyplus is at version 9.0. This version has only IDF files in the ExampleFolder. Everyone is still using IDF files. Does this mean I cannot use eppy3000 for any real work ?

If you can read and write IDF files from eppy3000, then eppy3000 would be usable. Here is how you can read an IDF file:

In [23]:
from eppy3000 import idfjsonconverter

fname = "../eppy3000/resources/snippets/V9_0/5Zone_Unitary_HXAssistedCoil.idf"
schemapath = "/Applications/EnergyPlus-9-0-1/Energy+.schema.epJSON"
idfhandle = open(fname, 'r')

jsonstr = idfjsonconverter.idf2json(idfhandle, open(schemapath, 'r'))
jsonfile = "5Zone_Unitary_HXAssistedCoil.json"
open(jsonfile, 'w').write(jsonstr)

148653

In [24]:
jsonhandle = open(jsonfile, 'r')
idfcopystr = idfjsonconverter.json2idf(jsonhandle, open(schemapath, 'r'))
outfile = "5Zone_Unitary_HXAssistedCoil_copy.idf"
open(outfile, 'w').write(idfcopystr)

169635